In [ ]:
#!pip install torch
!pip install sklearn
import torch.nn as nn
import sklearn
import torch
import pickle
import os

In [2]:
class Classifier(nn.Module):
    def __init__(self, hidden_size):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()

    def forward(self, x):#, mask_cls):
        h = self.linear1(x).squeeze(-1)
        sent_scores = self.relu(h) #* mask_cls.float()
        return sent_scores

def select_sent(phrase,clss,k=3):
    index_phrase=torch.topk(phrase,k)[1]
    pred_phrase=torch.zeros(clss.shape)
    index_1=[[clss[k].tolist().index(int(index_phrase[k][i])) 
                for i in range(len(index_phrase[k])) if int(index_phrase[k][i]) in clss[k].tolist()]
                 for k in range(k)]
    index_2=[[i] for i in range(k)]
    pred_phrase[index_2,index_1]=torch.ones(index_phrase.shape)
    return pred_phrase


from sklearn.metrics import confusion_matrix
def confusion_output(sent,output,clss_index):
    a=[confusion_matrix(output[i],sent[i]) for i in range(sent.shape[0])]
    c=[a[i][0][0]+a[i][1][1] for i in range(sent.shape[0])]
    score_total=[round((c[i]-(512-clss_index[i]))/(clss_index[i]),3) for i in range(sent.shape[0])]
    score_1=[(a[i][1][1])/3 for i in range(sent.shape[0])]
    return score_total,score_1

def Loss(x):
    c=np.mean([i-1 for i in x])
    return c

class Summarizer(nn.Module):
    def __init__(self, device):#args, , load_pretrained_bert = False, bert_config = None):
        super(Summarizer, self).__init__()
        self.device = device
        self.bert =CamembertModel.from_pretrained("camembert-base")
        #BertModel.from_pretrained('bert-base-uncased')
        #Bert(args.temp_dir, load_pretrained_bert, bert_config)
        self.encoder = Classifier(self.bert.config.hidden_size)
        self.select_sent=select_sent
        self.loss=nn.functional.binary_cross_entropy
        # self.score=confusion_output
        self.to(device)

    def forward(self,x,mask, mask_cls,clss,output,k=3):#,segs):#, sentence_range=None): #segs, 
        #x input_ids
        #Segs = Segment pour phrases (0 ou 1), marche pas dans un RoBERTa
        #clss index du début des phrases 
        #mask_cls vecteur pour passer de l'embedding au cls, en gros sélectionne le bon index des vecteurs de l'embedding qu'on va utiliser pour faire la classif
        top_vec= self.bert(x, mask)#, segs)
        # sents_vec=self.sent_vec(last,clss)

        # sents_vec = top_vec[0][torch.arange(top_vec[0].size(0)).unsqueeze(1), clss]
        # sents_vec = sents_vec * mask_cls[:, :, None].float()
        sent_scores = self.encoder(top_vec.last_hidden_state).squeeze(-1)#, mask_cls
        sent_scores_masked = torch.mul(sent_scores,mask_cls)
        sent_scores_masked = self.select_sent(sent_scores_masked,clss,k=k)
        
        # score=self.score(sent_scores,output)
        return sent_scores_masked,sent_scores,top_vec.last_hidden_state#,score#, mask_cls

In [ ]:
!pip install transformers

In [3]:
from transformers import CamembertModel,BertModel,RobertaModel,AdamW
camem=CamembertModel.from_pretrained("camembert-base")
camem.config.hidden_size

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


768

In [4]:
torch.cuda.is_available()

True

In [5]:
device = torch.device("cuda")
#cuda = torch.device('cuda')     # Default CUDA device
#cuda0 = torch.device('cuda:0')
#cuda2 = torch.device('cuda:2')
#cuda3 = torch.device('cuda:3')

#torch.rand(10, device=device)

In [6]:
train_dataset=pickle.load(open('train.pickle','rb'))

In [7]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size=8

dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size)

In [19]:
del summa,input_id,clss,mask_cls,output,mask
torch.cuda.empty_cache()

In [8]:
device = torch.device("cuda")
training_stats = []
epochs=3
summa=Summarizer(device=device)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0) 
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
f

14348994048

In [9]:
optimizer = AdamW(summa.parameters(),
                  lr = 2e-5, # Learning Rate
                  eps = 1e-8)# Epsilon

# Pour enregistrer les stats a chaque epoque
training_stats = []
perte=torch.nn.L1Loss()

In [10]:
# Boucle d'entrainement
for epoch in range(0, epochs):
     
    print("")
    print(f'########## Epoch {epoch+1} / {epochs} ##########')
    print('Training...')
 
 
    # On initialise la loss pour cette epoque
    total_train_loss = 0
 
    # On met le modele en mode 'training'
    # Dans ce mode certaines couches du modele agissent differement
    summa.train()
 
    # Pour chaque batch
    for step, batch in enumerate(dataloader):
 
        # On fait un print chaque 40 batchs
        if step % 40 == 0 and not step == 0:
            print(f'  Batch {step}  of {len(train_dataloader)}.')
         
        # On recupere les donnees du batch
        input_id = batch[0].to(device)
        mask = batch[1].to(device)
        clss = batch[2].to(device)
        mask_cls=batch[3].to(device)
        output=batch[4].to(device)
 
        # On met le gradient a 0
        summa.zero_grad()        
 
        # On passe la donnee au model et on recupere la loss et le logits (sortie avant fonction d'activation)
        sortie = summa(x=input_id,mask=mask,clss=clss,mask_cls=mask_cls,output=output)
        #score,score_1=confusion_output()
        loss=perte(sortie[0],output)
        # On incremente la loss totale
        # .item() donne la valeur numerique de la loss
        total_train_loss += loss.item()
 
        # Backpropagtion
        loss.backward()
 
        # On actualise les parametrer grace a l'optimizer
        optimizer.step()
 
    # On calcule la  loss moyenne sur toute l'epoque
    avg_train_loss = total_train_loss / len(train_dataloader)   
 
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))  
     
    # Enregistrement des stats de l'epoque
    training_stats.append(
        {
            'epoch': epoch + 1,
            'Training Loss': avg_train_loss,
        }
    )

print("Model saved!")
torch.save(summa.state_dict(), chemin_d+"model_essai.pt")



########## Epoch 1 / 3 ##########
Training...


ValueError: expected sequence of length 1 at dim 1 (got 3)

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [46]:
clss[0].tolist()

[0.0,
 32.0,
 55.0,
 71.0,
 73.0,
 102.0,
 107.0,
 119.0,
 142.0,
 149.0,
 153.0,
 179.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0